In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [3]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [4]:
data = pd.read_csv('201901-202003.csv')
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [5]:
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [6]:
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057394 entries, 0 to 1057393
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   CARD_SIDO_NM  1057394 non-null  object
 1   STD_CLSS_NM   1057394 non-null  object
 2   HOM_SIDO_NM   1057394 non-null  object
 3   AGE           1057394 non-null  object
 4   SEX_CTGO_CD   1057394 non-null  int64 
 5   FLC           1057394 non-null  int64 
 6   year          1057394 non-null  int64 
 7   month         1057394 non-null  int64 
 8   CSTMR_CNT     1057394 non-null  int64 
 9   AMT           1057394 non-null  int64 
 10  CNT           1057394 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 88.7+ MB


In [7]:
df_distance = df.copy()
df_distance['SIDOCHAI'] = 0

In [8]:
a = df_distance['CARD_SIDO_NM'] != df_distance['HOM_SIDO_NM']
df_distance.loc[a, 'SIDOCHAI'] = 1

In [83]:
df_distance.corr()

,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT,SIDOCHAI
SEX_CTGO_CD,1.000000,-0.009770,-0.008602,0.003076,-0.002302,-0.012303,-0.006518,-0.035615
FLC,-0.009770,1.000000,-0.007005,0.000993,0.001313,0.031418,-0.003896,0.001705
year,-0.008602,-0.007005,1.000000,-0.491575,-0.000396,0.001316,0.000437,-0.022558
month,0.003076,0.000993,-0.491575,1.000000,0.001512,0.000587,0.001937,0.011677
CSTMR_CNT,-0.002302,0.001313,-0.000396,0.001512,1.000000,0.861320,0.946729,-0.202935
AMT,-0.012303,0.031418,0.001316,0.000587,0.861320,1.000000,0.768612,-0.195416
CNT,-0.006518,-0.003896,0.000437,0.001937,0.946729,0.768612,1.000000,-0.168643
SIDOCHAI,-0.035615,0.001705,-0.022558,0.011677,-0.202935,-0.195416,-0.168643,1.000000


In [9]:
import scipy.stats as stats

In [10]:
corr = stats.spearmanr(df_distance['AMT'], df_distance['SIDOCHAI'])

In [11]:
cor = stats.spearmanr(df['AMT'], df['CSTMR_CNT'])
cor

SpearmanrResult(correlation=0.9119734964579476, pvalue=0.0)

In [12]:
df_distance

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT,SIDOCHAI
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,4,311200,4,0
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,3,605000,3,0
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,3,139000,3,0
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,3,27500,3,0
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,3,395500,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1057389,충북,휴양콘도 운영업,충북,70s,2,5,2019,3,3,148000,4,0
1057390,충북,휴양콘도 운영업,충북,70s,2,5,2019,5,5,329800,7,0
1057391,충북,휴양콘도 운영업,충북,70s,2,5,2019,10,7,557800,7,0
1057392,충북,휴양콘도 운영업,충북,70s,2,5,2019,12,3,247800,3,0


In [13]:
df_distance['PERSON'] = 0

In [14]:
b = df_distance['CSTMR_CNT']>df_distance['CNT']
c = df_distance['CSTMR_CNT']<=df_distance['CNT']

In [15]:
df_distance.loc[b,'PERSON'] = df_distance['CNT']
df_distance.loc[c,'PERSON'] = df_distance['CSTMR_CNT']

In [16]:
df_distance.drop(['HOM_SIDO_NM', 'CSTMR_CNT'], inplace = True, axis=1)

In [17]:
scorr = stats.spearmanr(df_distance['AMT'], df_distance['SIDOCHAI'])
scorr1 = stats.spearmanr(df_distance['AMT'], df_distance['PERSON'])
scorr2 = stats.spearmanr(df_distance['AMT'], df_distance['CNT'])

corr = stats.pearsonr(df_distance['AMT'], df_distance['SIDOCHAI'])
corr1 = stats.pearsonr(df_distance['AMT'], df_distance['PERSON'])
corr2 = stats.pearsonr(df_distance['AMT'], df_distance['CNT'])

scor = stats.spearmanr(df['AMT'], df['CSTMR_CNT'])
cor = stats.pearsonr(df['AMT'], df['CSTMR_CNT'])

In [18]:
print(scorr, scorr1, scorr2, scor) # 0 : 시도차이 / 1 : 인원조정(사람>건수 : 건수 / 사람 < 건수 : 사람) / 2. 기존값 / 3. drop값
print(corr, corr1, corr2, cor)

SpearmanrResult(correlation=-0.40045765875121137, pvalue=0.0) SpearmanrResult(correlation=0.9114314175847126, pvalue=0.0) SpearmanrResult(correlation=0.9065065697912096, pvalue=0.0) SpearmanrResult(correlation=0.9119734964579476, pvalue=0.0)
(-0.1954159557769239, 0.0) (0.8613189797042283, 0.0) (0.7686117577298156, 0.0) (0.8613202767167112, 0.0)


In [19]:
r1 = stats.spearmanr(df_distance['AMT'], df_distance['CARD_SIDO_NM']) #0.036
r2 = stats.spearmanr(df_distance['AMT'], df_distance['STD_CLSS_NM']) #0.039
r3 = stats.spearmanr(df_distance['AMT'], df_distance['AGE']) #0.077
r4 = stats.spearmanr(df_distance['AMT'], df_distance['SEX_CTGO_CD']) # 0.039
r5 = stats.spearmanr(df_distance['AMT'], df_distance['FLC']) # 0.070
r6 = stats.spearmanr(df_distance['AMT'], df_distance['year']) # 0.003
r7 = stats.spearmanr(df_distance['AMT'], df_distance['month']) # 0.001
print(r1, r2, r3, r4, r5, r6, r7)

SpearmanrResult(correlation=-0.036421369290906, pvalue=3.5068377010841654e-307) SpearmanrResult(correlation=0.03929796437012543, pvalue=0.0) SpearmanrResult(correlation=0.07767850535549146, pvalue=0.0) SpearmanrResult(correlation=-0.03902121935507375, pvalue=0.0) SpearmanrResult(correlation=0.07056506583311117, pvalue=0.0) SpearmanrResult(correlation=0.0031699053268085644, pvalue=0.0011156654561188019) SpearmanrResult(correlation=0.0011734906475679163, pvalue=0.22754943643730424)


In [20]:
df_distance

,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,year,month,AMT,CNT,SIDOCHAI,PERSON
0,강원,건강보조식품 소매업,20s,1,1,2019,1,311200,4,0,4
1,강원,건강보조식품 소매업,20s,1,1,2019,2,605000,3,0,3
2,강원,건강보조식품 소매업,20s,1,1,2019,6,139000,3,0,3
3,강원,건강보조식품 소매업,20s,1,1,2019,8,27500,3,0,3
4,강원,건강보조식품 소매업,20s,1,1,2019,9,395500,3,0,3
...,...,...,...,...,...,...,...,...,...,...,...
1057389,충북,휴양콘도 운영업,70s,2,5,2019,3,148000,4,0,3
1057390,충북,휴양콘도 운영업,70s,2,5,2019,5,329800,7,0,5
1057391,충북,휴양콘도 운영업,70s,2,5,2019,10,557800,7,0,7
1057392,충북,휴양콘도 운영업,70s,2,5,2019,12,247800,3,0,3
